In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Load the data
FirstData = np.load("Withbinaries.npy", allow_pickle=True)

# Split the data into features and labels
X = FirstData[:, 1:-1]  # All columns except the last one
y = FirstData[:, -1]   # The last column

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 40)
        self.fc2 = nn.Linear(40, 60)
        self.fc3 = nn.Linear(60, 50)
        self.fc4 = nn.Linear(50, 33)  # 33 units for 33 classes
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return self.softmax(x)

# Instantiate the model, define the loss function and the optimizer
model = NeuralNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training loop
for epoch in range(50):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, torch.max(labels, 1)[1])  # Use torch.max to convert one-hot to class indices
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        _, labels = torch.max(labels, 1)  # Convert one-hot to class indices
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

# Save the model
torch.save(model.state_dict(), "star_classifier_model.pth")


IndexError: Target 43 is out of bounds.

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Adjust optimizer learning rate if needed
optimizer = optim.Adam(model.parameters(), lr=0.005)

# Load the data
Withbinaries = np.load("Withbinaries.npy", allow_pickle=True)

# Split the data into features and labels
X = Withbinaries[:, 1:-1]  # All columns except the last one
y = Withbinaries[:, -1]    # The last column
ypreencode = y

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Adjusted Neural Network model
class NeuralNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 40)
        self.fc2 = nn.Linear(40, 60)
        self.fc3 = nn.Linear(60, 60)
        self.fc4 = nn.Linear(60, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)  # No softmax here
        return x

# Initialize model, loss, and optimizer
input_size = X_train.shape[1]
num_classes = len(np.unique(ypreencode))
model = NeuralNet(input_size, num_classes)

# Use cross-entropy loss (PyTorch expects class indices, not one-hot)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, torch.argmax(labels, dim=1))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy:.4f}')

# Save the model
torch.save(model.state_dict(), "star_classifier_withbin.pth")


Epoch [1/50], Loss: 3.7841
Epoch [2/50], Loss: 3.6855
Epoch [3/50], Loss: 3.7775
Epoch [4/50], Loss: 3.7679
Epoch [5/50], Loss: 3.6689
Epoch [6/50], Loss: 3.7273
Epoch [7/50], Loss: 3.7279
Epoch [8/50], Loss: 3.7707
Epoch [9/50], Loss: 3.7220
Epoch [10/50], Loss: 3.7444
Epoch [11/50], Loss: 3.7396
Epoch [12/50], Loss: 3.6959
Epoch [13/50], Loss: 3.7756
Epoch [14/50], Loss: 3.8408
Epoch [15/50], Loss: 3.7910
Epoch [16/50], Loss: 3.7251
Epoch [17/50], Loss: 3.7778
Epoch [18/50], Loss: 3.7278
Epoch [19/50], Loss: 3.7448
Epoch [20/50], Loss: 3.7644
Epoch [21/50], Loss: 3.6870
Epoch [22/50], Loss: 3.7695
Epoch [23/50], Loss: 3.7103
Epoch [24/50], Loss: 3.7182
Epoch [25/50], Loss: 3.8322
Epoch [26/50], Loss: 3.7648
Epoch [27/50], Loss: 3.7694
Epoch [28/50], Loss: 3.7446
Epoch [29/50], Loss: 3.7336
Epoch [30/50], Loss: 3.7083
Epoch [31/50], Loss: 3.8176
Epoch [32/50], Loss: 3.7600
Epoch [33/50], Loss: 3.7129
Epoch [34/50], Loss: 3.7144
Epoch [35/50], Loss: 3.7489
Epoch [36/50], Loss: 3.7176
E

In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from torch.utils.data import DataLoader, TensorDataset

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the data
FirstData = np.load("FirstData.npy", allow_pickle=True)

# Split the data into features and labels
X = FirstData[:, 1:-1]  # All columns except the last one
y = FirstData[:, -1]    # The last column
ypreencode = y

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))
ypostencode = y

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# Create PyTorch Datasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 40)
        self.fc2 = nn.Linear(40, 60)
        self.fc3 = nn.Linear(60, 50)
        self.fc4 = nn.Linear(50, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.softmax(self.fc4(x))
        return x

# Initialize the model, loss function, and optimizer
input_size = X_train.shape[1]
output_size = len(np.unique(ypreencode))
model = NeuralNetwork(input_size, output_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs.data, 1)
        _, labels = torch.max(y_batch.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy:.4f}')

# Save the model
torch.save(model.state_dict(), "star_classifier_model.pth")


Epoch 1/50, Loss: 2.9142
Epoch 2/50, Loss: 2.8454
Epoch 3/50, Loss: 2.8396
Epoch 4/50, Loss: 2.8363
Epoch 5/50, Loss: 2.8351
Epoch 6/50, Loss: 2.8345
Epoch 7/50, Loss: 2.8345
Epoch 8/50, Loss: 2.8326
Epoch 9/50, Loss: 2.8320
Epoch 10/50, Loss: 2.8334
Epoch 11/50, Loss: 2.8317
Epoch 12/50, Loss: 2.8320
Epoch 13/50, Loss: 2.8310
Epoch 14/50, Loss: 2.8313
Epoch 15/50, Loss: 2.8309
Epoch 16/50, Loss: 2.8292
Epoch 17/50, Loss: 2.8302
Epoch 18/50, Loss: 2.8305
Epoch 19/50, Loss: 2.8278
Epoch 20/50, Loss: 2.8295
Epoch 21/50, Loss: 2.8319
Epoch 22/50, Loss: 2.7993
Epoch 23/50, Loss: 2.7949
Epoch 24/50, Loss: 2.7922
Epoch 25/50, Loss: 2.7926
Epoch 26/50, Loss: 2.7931
Epoch 27/50, Loss: 2.7916
Epoch 28/50, Loss: 2.7920
Epoch 29/50, Loss: 2.7901
Epoch 30/50, Loss: 2.7895
Epoch 31/50, Loss: 2.7898
Epoch 32/50, Loss: 2.7913
Epoch 33/50, Loss: 2.7898
Epoch 34/50, Loss: 2.7888
Epoch 35/50, Loss: 2.7882
Epoch 36/50, Loss: 2.7897
Epoch 37/50, Loss: 2.7873
Epoch 38/50, Loss: 2.7898
Epoch 39/50, Loss: 2.

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm  # Import tqdm for the progress bar

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the data
FirstData = np.load("FirstData.npy", allow_pickle=True)

# Split the data into features and labels
X = FirstData[:, 1:-1]  # All columns except the last one
y = FirstData[:, -1]    # The last column
ypreencode = y

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))
ypostencode = y

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# Create PyTorch Datasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 40)
        self.fc2 = nn.Linear(40, 60)
        self.fc3 = nn.Linear(60, 50)
        self.fc4 = nn.Linear(50, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.softmax(self.fc4(x))
        return x

# Initialize the model, loss function, and optimizer
input_size = X_train.shape[1]
output_size = len(np.unique(ypreencode))
model = NeuralNetwork(input_size, output_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to calculate accuracy
def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(y_batch.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Training the model with a loading bar
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    # Use tqdm for a loading bar
    with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{epochs}', unit='batch') as pbar:
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            pbar.set_postfix({'Loss': f'{running_loss/len(train_loader):.4f}'})
            pbar.update(1)
    
    # Calculate and print the test accuracy after each epoch
    test_accuracy = calculate_accuracy(test_loader, model)
    print(f'\nTest Accuracy after Epoch {epoch+1}: {test_accuracy:.4f}')

# Save the model
torch.save(model.state_dict(), "star_classifier_model.pth")


Epoch 1/50: 100%|██████████| 1985/1985 [00:07<00:00, 252.68batch/s, Loss=2.8836]



Test Accuracy after Epoch 1: 0.6991


Epoch 2/50: 100%|██████████| 1985/1985 [00:07<00:00, 268.11batch/s, Loss=2.8432]



Test Accuracy after Epoch 2: 0.7051


Epoch 3/50: 100%|██████████| 1985/1985 [00:07<00:00, 267.26batch/s, Loss=2.8396]


...


Test Accuracy after Epoch 48: 0.7648


Epoch 49/50: 100%|██████████| 1985/1985 [00:08<00:00, 237.16batch/s, Loss=2.7763]



Test Accuracy after Epoch 49: 0.7640


Epoch 50/50: 100%|██████████| 1985/1985 [00:08<00:00, 232.28batch/s, Loss=2.7753]



Test Accuracy after Epoch 50: 0.7735


In [9]:
# Evaluate the model and get predictions
model.eval()
_, all_preds, all_labels = calculate_accuracy(test_loader, model)

# Confusion matrix and classification report
conf_matrix = confusion_matrix(all_labels, all_preds)
class_report = classification_report(all_labels, all_preds, target_names=[str(i) for i in range(output_size)])

print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(class_report)

# Save the model
torch.save(model.state_dict(), "star_classifier_model.pth")

TypeError: cannot unpack non-iterable float object